In [ ]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz, process
import re
import json

# Sample DataFrame setup
data = {
    'Ticket': range(1, 21),
    'Business Service': [
        'Microsoft Outlook', 'Microsoft Windows', 'Laptop', 'Python',
        'voice log', 'crm service', 'cwhh vdi', 'vantage agent portal',
        'Microsoft Excel', 'MS Excel', 'Excel 365', 'outlook email',
        'Outlook client', 'Windows 10', 'Microsoft Windows 11',
        'laptop hardware', 'laptop support', 'python development',
        'Python scripting', 'CRM system'
    ]
}
df = pd.DataFrame(data)

# Method 1: LLM-based approach
def llm_based_standardization(df, chat_function):
    """
    Use LLM to group similar services and standardize them
    """
    # Get unique services and their counts
    service_counts = df['Business Service'].value_counts()
    unique_services = service_counts.index.tolist()
    
    # Create prompt for LLM to group similar services
    prompt = f"""
    I have a list of business services that need to be standardized. Please group similar services together and for each group, suggest the most representative name (preferably the one that appears most frequently).

    Services with their counts:
    {dict(service_counts)}

    Please return a JSON object where:
    - Keys are the standardized service names
    - Values are lists of all variations that should map to that standardized name

    Example format:
    {{
        "Microsoft Excel": ["Microsoft Excel", "MS Excel", "Excel 365"],
        "Microsoft Outlook": ["Microsoft Outlook", "outlook email", "Outlook client"]
    }}

    Focus on grouping services that refer to the same underlying technology or service.
    """
    
    try:
        # Call the LLM function
        response = chat_function(prompt)
        
        # Parse the JSON response
        # Note: You might need to extract JSON from the response text
        standardization_map = json.loads(response)
        
        # Create a mapping from original to standardized
        service_mapping = {}
        for standard_name, variations in standardization_map.items():
            for variation in variations:
                service_mapping[variation] = standard_name
        
        # Apply mapping to create new column
        df['New Services'] = df['Business Service'].map(
            lambda x: service_mapping.get(x, x)
        )
        
        return df, service_mapping
        
    except Exception as e:
        print(f"Error in LLM processing: {e}")
        return df, {}

# Method 2: Traditional NLP approach
def traditional_nlp_standardization(df, similarity_threshold=80):
    """
    Use fuzzy matching and pattern recognition to standardize services
    """
    # Get service counts
    service_counts = df['Business Service'].value_counts()
    unique_services = list(service_counts.index)
    
    # Preprocessing function
    def preprocess_service(service):
        # Convert to lowercase, remove extra spaces, common prefixes/suffixes
        service = service.lower().strip()
        service = re.sub(r'\bmicrosoft\b|\bms\b', 'microsoft', service)
        service = re.sub(r'\b365\b|\boffice\b', '', service).strip()
        service = re.sub(r'\s+', ' ', service)
        return service
    
    # Create groups of similar services
    groups = []
    used_services = set()
    
    for service in unique_services:
        if service in used_services:
            continue
            
        # Find similar services
        similar_services = [service]
        used_services.add(service)
        
        for other_service in unique_services:
            if other_service in used_services:
                continue
                
            # Check similarity using multiple methods
            similarity_scores = [
                fuzz.ratio(preprocess_service(service), preprocess_service(other_service)),
                fuzz.partial_ratio(service.lower(), other_service.lower()),
                fuzz.token_sort_ratio(service.lower(), other_service.lower()),
                fuzz.token_set_ratio(service.lower(), other_service.lower())
            ]
            
            max_similarity = max(similarity_scores)
            
            # Also check for keyword overlap
            service_words = set(preprocess_service(service).split())
            other_words = set(preprocess_service(other_service).split())
            word_overlap = len(service_words & other_words) / max(len(service_words), len(other_words))
            
            if max_similarity >= similarity_threshold or word_overlap >= 0.6:
                similar_services.append(other_service)
                used_services.add(other_service)
        
        if len(similar_services) > 1:
            groups.append(similar_services)
    
    # For each group, find the service with highest count
    service_mapping = {}
    for group in groups:
        # Get counts for services in this group
        group_counts = {service: service_counts[service] for service in group}
        # Find the service with maximum count
        standard_service = max(group_counts, key=group_counts.get)
        
        # Map all services in group to the standard one
        for service in group:
            service_mapping[service] = standard_service
    
    # Apply mapping
    df['New Services'] = df['Business Service'].map(
        lambda x: service_mapping.get(x, x)
    )
    
    return df, service_mapping, groups

# Method 3: Hybrid approach
def hybrid_standardization(df, chat_function, similarity_threshold=75):
    """
    Combine traditional NLP for initial grouping and LLM for validation/refinement
    """
    # Step 1: Use traditional NLP for initial grouping
    df_temp, initial_mapping, groups = traditional_nlp_standardization(
        df.copy(), similarity_threshold
    )
    
    # Step 2: Use LLM to validate and refine the groups
    if groups:
        prompt = f"""
        I've used fuzzy matching to group similar business services. Please review these groups and suggest improvements:

        Groups found:
        {json.dumps(groups, indent=2)}

        Service counts:
        {dict(df['Business Service'].value_counts())}

        Please return a JSON object with refined groupings where:
        - Keys are the best representative names for each service category
        - Values are lists of all variations that should map to that name
        
        Validate that the groupings make sense semantically.
        """
        
        try:
            response = chat_function(prompt)
            refined_mapping = json.loads(response)
            
            # Create final mapping
            final_service_mapping = {}
            for standard_name, variations in refined_mapping.items():
                for variation in variations:
                    final_service_mapping[variation] = standard_name
            
            df['New Services'] = df['Business Service'].map(
                lambda x: final_service_mapping.get(x, x)
            )
            
            return df, final_service_mapping
            
        except Exception as e:
            print(f"Error in LLM refinement: {e}")
            return df_temp, initial_mapping
    
    return df_temp, initial_mapping

# Example usage and comparison
def compare_methods(df):
    """
    Compare the effectiveness of different methods
    """
    print("Original services:")
    print(df['Business Service'].value_counts())
    print("\n" + "="*50 + "\n")
    
    # Traditional NLP method
    df_traditional, mapping_traditional, groups = traditional_nlp_standardization(df.copy())
    print("Traditional NLP Results:")
    print("Groups found:", groups)
    print("Standardized services:")
    print(df_traditional['New Services'].value_counts())
    print(f"Reduced from {df['Business Service'].nunique()} to {df_traditional['New Services'].nunique()} unique services")
    
    return df_traditional, mapping_traditional

# Mock chat function for testing (replace with your actual LLM endpoint)
def mock_chat_function(prompt):
    """
    Mock LLM function - replace this with your actual chat function
    """
    # This is a mock response - your actual LLM should analyze the prompt
    mock_response = '''
    {
        "Microsoft Excel": ["Microsoft Excel", "MS Excel", "Excel 365"],
        "Microsoft Outlook": ["Microsoft Outlook", "outlook email", "Outlook client"],
        "Microsoft Windows": ["Microsoft Windows", "Windows 10", "Microsoft Windows 11"],
        "Laptop": ["Laptop", "laptop hardware", "laptop support"],
        "Python": ["Python", "python development", "Python scripting"],
        "CRM Service": ["crm service", "CRM system"],
        "Voice Log": ["voice log"],
        "CWHH VDI": ["cwhh vdi"],
        "Vantage Agent Portal": ["vantage agent portal"]
    }
    '''
    return mock_response

# Run comparison
if __name__ == "__main__":
    # Test traditional method
    df_result, mapping = compare_methods(df)
    
    print("\n" + "="*50 + "\n")
    print("Final DataFrame with standardized services:")
    print(df_result[['Business Service', 'New Services']].head(10))
    
    # Test LLM method (uncomment when you have actual chat function)
    # df_llm, mapping_llm = llm_based_standardization(df.copy(), your_chat_function)
    # print("LLM Results:", df_llm['New Services'].value_counts())

In [ ]:
import pandas as pd
import json
import re
from typing import Dict, Tuple, List

def llm_service_standardization(df: pd.DataFrame, chat_function, 
                              service_column: str = 'Business Service',
                              new_column: str = 'New Services') -> Tuple[pd.DataFrame, Dict]:
    """
    Standardize business services using LLM with robust error handling
    
    Args:
        df: DataFrame containing the service data
        chat_function: Your LLM chat function
        service_column: Name of column containing services to standardize
        new_column: Name of new column for standardized services
    
    Returns:
        Tuple of (updated_dataframe, service_mapping_dict)
    """
    
    # Get service counts
    service_counts = df[service_column].value_counts()
    
    # Create the prompt
    prompt = create_standardization_prompt(service_counts)
    
    try:
        # Call your LLM
        print("Calling LLM for service standardization...")
        response = chat_function(prompt)
        print(f"LLM Response received: {len(response)} characters")
        
        # Parse the response
        standardization_map = parse_llm_response(response)
        
        # Validate the mapping
        standardization_map = validate_mapping(standardization_map, service_counts.index.tolist())
        
        # Apply the mapping
        df = apply_service_mapping(df, standardization_map, service_column, new_column)
        
        # Print results
        print_results(df, service_column, new_column, standardization_map)
        
        return df, standardization_map
        
    except Exception as e:
        print(f"Error in LLM standardization: {e}")
        # Fallback: copy original column
        df[new_column] = df[service_column]
        return df, {}

def create_standardization_prompt(service_counts) -> str:
    """Create a detailed prompt for the LLM"""
    
    services_text = "\n".join([f"- {service}: {count} occurrences" 
                              for service, count in service_counts.items()])
    
    prompt = f"""
You are helping standardize business service names in a IT ticketing system. 

TASK: Group similar services together and choose the best representative name for each group.

SERVICES AND THEIR FREQUENCIES:
{services_text}

RULES:
1. Group services that refer to the same underlying technology/service
2. For each group, choose the name with the HIGHEST frequency as the standard
3. If frequencies are equal, choose the most descriptive/official name
4. Keep unrelated services separate
5. Be conservative - only group if you're confident they're the same service

EXAMPLES of what should be grouped:
- "Microsoft Excel", "MS Excel", "Excel 365" → all refer to Excel
- "Microsoft Outlook", "Outlook email", "Outlook client" → all refer to Outlook
- "Windows 10", "Microsoft Windows 11", "Microsoft Windows" → all refer to Windows OS

EXAMPLES of what should NOT be grouped:
- "Python" and "Java" → different programming languages
- "Laptop" and "Desktop" → different hardware types
- "CRM" and "ERP" → different software categories

OUTPUT FORMAT:
Return ONLY a valid JSON object with this exact structure:
{{
    "Standard Service Name 1": ["variation1", "variation2", "variation3"],
    "Standard Service Name 2": ["variation1", "variation2"],
    "Ungrouped Service": ["Ungrouped Service"]
}}

IMPORTANT: 
- Include ALL original services in your response
- Each service should appear exactly once
- Use the exact service names from the input list
- Return only the JSON, no additional text
"""
    
    return prompt

def parse_llm_response(response: str) -> Dict:
    """Parse LLM response and extract JSON mapping"""
    
    # Try to find JSON in the response
    json_match = re.search(r'\{.*\}', response, re.DOTALL)
    
    if json_match:
        json_str = json_match.group()
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            print(f"Problematic JSON: {json_str[:200]}...")
    
    # If direct parsing fails, try to clean the response
    try:
        # Remove markdown code blocks if present
        cleaned = re.sub(r'```json\s*|\s*```', '', response)
        cleaned = re.sub(r'```\s*|\s*```', '', cleaned)
        
        # Find the JSON object
        start = cleaned.find('{')
        end = cleaned.rfind('}') + 1
        
        if start != -1 and end > start:
            json_str = cleaned[start:end]
            return json.loads(json_str)
            
    except Exception as e:
        print(f"Error cleaning response: {e}")
    
    raise ValueError("Could not parse valid JSON from LLM response")

def validate_mapping(mapping: Dict, original_services: List[str]) -> Dict:
    """Validate and fix the LLM mapping"""
    
    # Flatten all mapped services
    mapped_services = set()
    for variations in mapping.values():
        mapped_services.update(variations)
    
    # Find missing services
    original_set = set(original_services)
    missing_services = original_set - mapped_services
    
    # Add missing services as standalone entries
    for service in missing_services:
        mapping[service] = [service]
        print(f"Added missing service: {service}")
    
    # Remove any services not in original list
    cleaned_mapping = {}
    for standard_name, variations in mapping.items():
        valid_variations = [v for v in variations if v in original_set]
        if valid_variations:
            cleaned_mapping[standard_name] = valid_variations
    
    return cleaned_mapping

def apply_service_mapping(df: pd.DataFrame, mapping: Dict, 
                         service_column: str, new_column: str) -> pd.DataFrame:
    """Apply the standardization mapping to the dataframe"""
    
    # Create reverse mapping: original_service -> standard_service
    service_map = {}
    for standard_name, variations in mapping.items():
        for variation in variations:
            service_map[variation] = standard_name
    
    # Apply mapping
    df[new_column] = df[service_column].map(service_map)
    
    # Handle any unmapped services (shouldn't happen with validation)
    unmapped_mask = df[new_column].isna()
    if unmapped_mask.any():
        print(f"Warning: {unmapped_mask.sum()} services couldn't be mapped")
        df.loc[unmapped_mask, new_column] = df.loc[unmapped_mask, service_column]
    
    return df

def print_results(df: pd.DataFrame, original_col: str, new_col: str, mapping: Dict):
    """Print standardization results"""
    
    print("\n" + "="*60)
    print("STANDARDIZATION RESULTS")
    print("="*60)
    
    original_unique = df[original_col].nunique()
    new_unique = df[new_col].nunique()
    reduction = original_unique - new_unique
    
    print(f"Original unique services: {original_unique}")
    print(f"Standardized unique services: {new_unique}")
    print(f"Reduction: {reduction} services ({reduction/original_unique*100:.1f}%)")
    
    print("\nGROUPINGS MADE:")
    for standard_name, variations in mapping.items():
        if len(variations) > 1:
            print(f"\n'{standard_name}' ← {variations}")
    
    print(f"\nFINAL SERVICE DISTRIBUTION:")
    print(df[new_col].value_counts().head(10))

# USAGE EXAMPLE WITH YOUR CHAT FUNCTION
def example_usage():
    """
    Example of how to use with your actual chat function
    """
    
    # Your DataFrame
    df = pd.DataFrame({
        'Ticket': range(1, 21),
        'Business Service': [
            'Microsoft Outlook', 'Microsoft Windows', 'Laptop', 'Python',
            'voice log', 'crm service', 'cwhh vdi', 'vantage agent portal',
            'Microsoft Excel', 'MS Excel', 'Excel 365', 'outlook email',
            'Outlook client', 'Windows 10', 'Microsoft Windows 11',
            'laptop hardware', 'laptop support', 'python development',
            'Python scripting', 'CRM system'
        ]
    })
    
    # Replace this with your actual chat function
    def your_chat_function(prompt):
        """
        Replace this with your actual LLM endpoint
        """
        # Example: return chat(prompt)
        # or: return openai.chat.completions.create(...)
        # or: return your_llm_api_call(prompt)
        
        # For now, using a mock response
        return '''
        {
            "Microsoft Excel": ["Microsoft Excel", "MS Excel", "Excel 365"],
            "Microsoft Outlook": ["Microsoft Outlook", "outlook email", "Outlook client"],
            "Microsoft Windows": ["Microsoft Windows", "Windows 10", "Microsoft Windows 11"],
            "Laptop": ["Laptop", "laptop hardware", "laptop support"],
            "Python": ["Python", "python development", "Python scripting"],
            "CRM service": ["crm service", "CRM system"],
            "voice log": ["voice log"],
            "cwhh vdi": ["cwhh vdi"],
            "vantage agent portal": ["vantage agent portal"]
        }
        '''
    
    # Run the standardization
    df_result, mapping = llm_service_standardization(
        df=df,
        chat_function=your_chat_function,  # Replace with your actual function
        service_column='Business Service',
        new_column='New Services'
    )
    
    return df_result, mapping

# INTEGRATION PATTERNS FOR DIFFERENT LLM PROVIDERS

def openai_integration_example():
    """Example for OpenAI API"""
    import openai
    
    def chat_with_openai(prompt):
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1  # Low temperature for consistent results
        )
        return response.choices[0].message.content

def anthropic_integration_example():
    """Example for Anthropic Claude API"""
    import anthropic
    
    client = anthropic.Anthropic(api_key="your-api-key")
    
    def chat_with_claude(prompt):
        response = client.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=2000,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.content[0].text

def custom_endpoint_example():
    """Example for custom API endpoint"""
    import requests
    
    def chat_with_custom_api(prompt):
        response = requests.post(
            "https://your-api-endpoint.com/chat",
            json={"prompt": prompt, "max_tokens": 2000},
            headers={"Authorization": "Bearer your-token"}
        )
        return response.json()["response"]

# Run the example
if __name__ == "__main__":
    df_result, mapping = example_usage()
    
    print("\nSample of final results:")
    print(df_result[['Business Service', 'New Services']].head(10))